<a href="https://colab.research.google.com/github/KimEunChae9506/2024-smartfarm-AI-strongberry/blob/main/%EC%82%AC%EC%A0%84%ED%85%8C%EC%8A%A4%ED%8A%B8_%ED%85%8C%EC%8A%A4%ED%8A%B8%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#조사항목값의 컬럼을 빼서 피봇테이블을 만든 후 환경데이터와 merge.
#하지만 원본과 MASE를 비교할 때 output csv와 형식이 맞지 않을 것 같아 제출X.

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import joblib

# 생육 데이터 불러오기
growth_file = '사전테스트-생육데이터.xlsx'

# DataFrame 형식으로 엑셀 파일 읽기
growth_data = pd.read_excel(growth_file)

# 환경 데이터 불러오기
farm_files = ['environmentsB.csv', 'environmentsC.csv', 'environmentsD.csv', 'environmentsE.csv']
environment_data = pd.DataFrame()

for file_name in farm_files:
    temp = pd.read_csv(file_name, sep=',', encoding='utf-8')
    environment_data = pd.concat([environment_data, temp])

# 생육 데이터 클리닝
growth_data['조사일자'] = pd.to_datetime(growth_data['조사일자'], format='%Y%m%d')

# 환경 데이터 클리닝
environment_data['datetime'] = pd.to_datetime(environment_data['datetime'])
environment_data = environment_data[~((environment_data['supplyEC'] == 0) & (environment_data['innerSolar'] == 0))]
environment_data = environment_data.dropna(subset=['datetime'])

# 생육 데이터 피봇 (조사항목 열의 값들을 컬럼으로 빼준다.)
pivoted_growth_data = growth_data.pivot_table(
    index=['시설아이디', '생육주사', '조사일자', '표본번호'],
    columns='조사항목',
    values='조사항목값'
).reset_index()

# 데이터 병합
environment_data.rename(columns={'farm': '시설아이디'}, inplace=True)
merged_data = pd.merge_asof(pivoted_growth_data.sort_values('조사일자'), environment_data.sort_values('datetime'),
                            by='시설아이디', left_on='조사일자', right_on='datetime', direction='nearest')

merged_data.drop(['조사일자', '시설아이디', '생육주사', '표본번호', '최종화방차수'], axis=1, inplace=True)

# 입력 변수(X)와 출력 변수(y) 분리
X = merged_data[['supplyEC', 'supplyPH', 'innerCO2', 'innerHum', 'innerTemp', 'innerSolar']]
y = merged_data[['초장', '엽장', '엽폭', '엽병장', '엽수', '관부직경', '화방 꽃수(소화수)', '착과수']]

# 데이터 스케일링
xMscaler = MinMaxScaler(feature_range=(-1,1))
yMscaler = MinMaxScaler(feature_range=(-1,1))
X = xMscaler.fit_transform(X)
y = yMscaler.fit_transform(y)

# 데이터를 훈련 세트와 테스트 세트로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# 모델 저장
joblib.dump(model, 'trained_model.pkl')

# 모델 불러오기
model = joblib.load('trained_model.pkl')

# 새로운 입력 데이터 불러오기
new_env_data = merged_data

# 예측 수행
predictions = model.predict(xMscaler.transform(new_env_data[['supplyEC', 'supplyPH', 'innerCO2', 'innerHum', 'innerTemp', 'innerSolar']]))

# 예측 결과를 데이터프레임으로 변환
predicted_growth = pd.DataFrame(yMscaler.inverse_transform(predictions), columns=['초장', '엽장', '엽폭', '엽병장', '엽수', '관부직경', '화방 꽃수(소화수)', '착과수'])

# 결과를 CSV 파일로 저장
predicted_growth.to_csv('predicted_growth.csv', index=False)

# MASE 계산 함수
def mase(actual, predicted, seasonality=1):
    n = len(actual)
    d = np.abs(np.diff(actual, n=seasonality)).sum() / (n - seasonality)
    errors = np.abs(actual - predicted)
    return errors.mean() / d

pivoted_growth_data.drop(['조사일자', '시설아이디', '생육주사', '표본번호', '최종화방차수'], axis=1, inplace=True)

# 각 열에 대해 MASE 계산
mase_scores = {}
for column in pivoted_growth_data.columns:
    actual = pivoted_growth_data[column].dropna().values
    predicted = predicted_growth[column].values[:len(actual)]  # 실제 값과 예측 값을 맞추기 위해 슬라이싱
    mase_score = mase(actual, predicted)
    mase_scores[column] = mase_score

# 결과 출력
print(mase_scores)
# 예측 및 평가
y_pred = model.predict(X_test)
y_pred = yMscaler.inverse_transform(y_pred)  # 스케일링 되돌리기
y_test = yMscaler.inverse_transform(y_test)  # 스케일링 되돌리기

{'관부직경': 1.43435689282243, '엽병장': 1.5275196831063476, '엽수': 1.3234630739025826, '엽장': 1.6597284355678907, '엽폭': 1.6356605555780686, '착과수': 2.0649050099393276, '초장': 1.5415704224810618, '화방 꽃수(소화수)': 1.9810629436925864}
